In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
import zipfile
import time
import shutil
from sklearn.metrics import log_loss
import zipcode

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """
    Use of this metric is not recommended; for illustration only. 
    See other regression metrics on sklearn docs:
      http://scikit-learn.org/stable/modules/classes.html#regression-metrics
    Use like any other metric
    >>> y_true = [3, -0.5, 2, 7]; y_pred = [2.5, -0.3, 2, 8]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    Out[]: 24.791666666666668
    """

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
from sklearn.metrics import make_scorer

In [4]:
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [5]:
train = pd.read_csv('Fr_train.csv', sep=';')
test = pd.read_csv('Fr_test.csv', sep=';')

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
target = train.prime_tot_ttc
#train = train.drop(['prime_tot_ttc'],axis=1)

In [7]:
post_city  = []
post_state = []
post_wage = []
post_population = []
post_tax = []
post_decom = []

for i in train.codepostal.values:
    
    try:
        state = zipcode.isequal(str(i)).state
        post_state.append(state)
    except:
        post_state.append('nothing')
        
    try:
        city = zipcode.isequal(str(i)).city
        post_city.append(city)
    except:
        post_city.append('nothing')
        
    try:
        wage = int(zipcode.isequal(str(i)).wages)
        post_wage.append(wage)
    except:
        post_wage.append(0)
        
    try:
        pop = int(zipcode.isequal(str(i)).population)
        post_population.append(pop)
    except:
        post_population.append(0)
        
    try:
        tax = int(zipcode.isequal(str(i)).tax_returns_filed)
        post_tax.append(tax)
    except:
        post_tax.append(0)
        
    try:
        decom = int(zipcode.isequal(str(i)).decommisioned)
        post_decom.append(decom)
    except:
        post_decom.append(0)

In [8]:
post_city_test  = []
post_state_test = []
post_wage_test = []
post_population_test = []
post_tax_test = []
post_decom_test = []

for i in test.codepostal.values:
    
    try:
        state = zipcode.isequal(str(i)).state
        post_state_test.append(state)
    except:
        post_state_test.append('no_state')
        
    try:
        city = zipcode.isequal(str(i)).city
        post_city_test.append(city)
    except:
        post_city_test.append('no_city')
        
    try:
        wage = int(zipcode.isequal(str(i)).wages)
        post_wage_test.append(wage)
    except:
        post_wage_test.append(0)
        
    try:
        pop = int(zipcode.isequal(str(i)).population)
        post_population_test.append(pop)
    except:
        post_population_test.append(0)
        
    try:
        tax = int(zipcode.isequal(str(i)).tax_returns_filed)
        post_tax_test.append(tax)
    except:
        post_tax_test.append(0)
        
    try:
        decom = int(zipcode.isequal(str(i)).decommisioned)
        post_decom_test.append(decom)
    except:
        post_decom_test.append(0)

In [9]:
test['state'] = post_state_test
#test['city'] = post_city_test
test['wage'] = post_wage_test
test['population'] = post_population_test

In [10]:
test = test.drop(['codepostal'],axis=1)

In [11]:
train['state'] = post_state
#train['city'] = post_city
train['wage'] = post_wage
train['population'] = post_population

In [12]:
train = train.drop(['codepostal'],axis=1)

In [13]:
train.head(2)

,id,annee_naissance,annee_permis,marque,puis_fiscale,anc_veh,energie_veh,kmage_annuel,crm,profession,...,var17,var18,var19,var20,var21,var22,prime_tot_ttc,state,wage,population
0,1,1986.0,2006.0,RENAULT,4,1,gpl,2924,68,"enseignant, formateur, chercheur",...,1,27,0,0,0,1,254.75,nothing,0,0
1,2,1986.0,2006.0,RENAULT,8,2,gpl,11580,50,"enseignant, formateur, chercheur",...,1,28,0,0,0,1,259.89,nothing,0,0


In [14]:
train = train.fillna(train.median())

In [15]:
test = test.fillna(test.median())

In [16]:
train.annee_naissance = train.annee_naissance.astype(int).astype(str)
train.annee_permis = train.annee_permis.astype(int).astype(str)
#train.puis_fiscale = train.puis_fiscale.astype(int).astype(str)

In [17]:
train_dummy = pd.get_dummies(train)

In [18]:
train_dummy.head(2)

,id,puis_fiscale,anc_veh,kmage_annuel,crm,var1,var2,var3,var4,var5,...,state_PA,state_SD,state_TN,state_TX,state_UT,state_VA,state_WI,state_WV,state_WY,state_nothing
0,1,4,1,2924,68,10.0,1,372,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,8,2,11580,50,10.0,1,372,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
test_dummy = pd.get_dummies(test)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

In [21]:
#score = []
#for i in range(210,260,10):
    #gb = GradientBoostingRegressor(n_estimators=i, max_depth=5, subsample=0.6, warm_start=True, verbose=1)
    #rf = RandomForestRegressor(n_estimators=i, warm_start=True, max_depth=16, verbose=1, n_jobs=8)
    #score.append(cross_val_score(gb, train_dummy[list(set(test_dummy.columns)&(set(train_dummy.columns)))], target, cv=3, n_jobs=8,scoring=scorer).mean())
    #print(i, score)

In [ ]:
gb = GradientBoostingRegressor(n_estimators=230, max_depth=5, subsample=0.5, verbose=1)
gb.fit(train_dummy[list(set(test_dummy.columns)&(set(train_dummy.columns)))], target)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       12337.8839        1891.5428           14.12m
         2       10918.4670        1549.3116           14.08m
         3        9742.7342        1256.3189           13.99m
         4        8610.1509        1052.9659           13.99m
         5        7747.2253         853.8435           13.99m
        80        1850.7772           5.2536            8.51m
        90        1792.7074           6.5048            7.83m
       100        1736.0688           2.1346            7.21m


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=5, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=230,
             presort='auto', random_state=None, subsample=0.5, verbose=1,
             warm_start=False)

In [ ]:
prediction = gb.predict(test_dummy[list(set(test_dummy.columns)&(set(train_dummy.columns)))])

In [ ]:
result = pd.DataFrame([test_dummy.id.values,prediction])
result = result.transpose()
cols = ['ID','COTIS']
result.columns = cols
result.ID = result.ID.astype(int)

In [ ]:
result.head()

In [ ]:
result.to_csv('result.csv',sep=';',index_label=False, index=False)

In [ ]:
cols = train_dummy.columns
importances = list(gb.feature_importances_)

In [ ]:
imp = pd.DataFrame([cols,importances])
imp = imp.transpose()
columns=['names','importances']
imp.columns = columns

In [ ]:
imp[imp.importances>0].sort_values(by=['importances'],ascending=False).head(50)